# Training Your Custom Dataset Keyword Spotting Model

It is now time for you to train your own custom keyword spotting model using your Custom Dataset!

#Setup

### Import packages
Clone the TensorFlow Github Repository, which contains the relevant code required to run this assignment.

In [1]:
!wget https://github.com/tensorflow/tensorflow/archive/v2.4.1.zip
!unzip v2.4.1.zip &> 0
!mv tensorflow-2.4.1/ tensorflow/

--2025-12-14 08:44:11--  https://github.com/tensorflow/tensorflow/archive/v2.4.1.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/tensorflow/tensorflow/zip/refs/tags/v2.4.1 [following]
--2025-12-14 08:44:11--  https://codeload.github.com/tensorflow/tensorflow/zip/refs/tags/v2.4.1
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v2.4.1.zip’

v2.4.1.zip              [     <=>            ]  66.13M  12.7MB/s    in 5.7s    

2025-12-14 08:44:17 (11.5 MB/s) - ‘v2.4.1.zip’ saved [69346072]



### Important - Reverting to TF1 - Start
We need to install Tensorflow V1 in order to use the required features for Keyword Spotting.

**Run the following cell and click "restart runtime" that appears once it completes.**

**Do not rerun the previous cells after restarting the runtime.**

In [5]:
!pip uninstall tensorflow -y
!pip install tensorflow-gpu==1.15.0

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==1.15.0 (from versions: 2.12.0)
ERROR: No matching distribution found for tensorflow-gpu==1.15.0


### Important - Reverting to TF1 - End

**Make sure to click the "RESTART RUNTIME" button in the output of the previous cell. Then move onto the next cell!**

In [6]:
import tensorflow.compat.v1 as tf
# assert(tf.VERSION == "1.15.0") #Make sure you install Tensorflow 1.15.0 and restart the runtime or this will fail!
# Now that the runtime is set import things!
import sys
# We add this path so we can import the speech processing modules.
sys.path.append("/content/tensorflow/tensorflow/examples/speech_commands/")
import input_data
import models
import numpy as np
import glob
import os
import re
import shutil
import subprocess
import tempfile
from google.colab import files
!pip install ffmpeg-python &> 0
!apt-get update && apt-get -qq install xxd

ModuleNotFoundError: No module named 'tensorflow.compat'

### Загрузка данных для предобучения
Подгрузим Pete's dataset, чтобы использовать в качестве данных для предобучения. Это будет стадия пре-трейна, которая позволит предварительно обучить модель для задачи KWS, а затем мы произведем fine-tuning на данных со словами 'не слышу', 'не слышно'. Потенциально, это позволит добиться более высокого качества.

Набор слов, которые детектируются в Pete's dataset: "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", “backward”, “forward”, “follow”, “learn”.

In [ ]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz
DATASET_DIR =  'dataset/'
!mkdir dataset
!tar -xf speech_commands_v0.02.tar.gz -C 'dataset'
!rm -r -f speech_commands_v0.02.tar.gz

Загрузим данные для задания

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tarfile
file_path = '/content/drive/MyDrive/SpeechTechnologies/train_data.tar'
with tarfile.open(file_path, 'r') as tar_train:
    tar_train.extractall()


In [ ]:
file_path = '/content/drive/MyDrive/SpeechTechnologies/test_data.tar'
with tarfile.open(file_path, 'r') as tar_test:
    tar_test.extractall()

In [ ]:
base_train_path = '/content/train_opus/audio'
json_train_path = '/content/train_opus/word_bounds.json'
base_test_path = '/test_opus/audio'

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!apt-get install -y ffmpeg

In [ ]:
!apt-get update && apt-get install -y ffmpeg

In [ ]:
import os
import json
import pandas as pd
import torch
import torchaudio
import librosa
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import gc
import random

TARGET_SR = 16000  # Стандарт для KWS
SEGMENT_DURATION = 2.0
SEGMENT_SAMPLES = int(SEGMENT_DURATION * TARGET_SR)
CONTEXT_RATIO = 0.3  # 30% контекста вокруг слова


def load_and_convert_opus_files(base_path, json_path, sample_files=None):
    """
    Загружает JSON и конвертирует OPUS файлы в аудио тензоры

    Args:
        base_path: путь к папке с OPUS файлами
        json_path: путь к JSON файлу с аннотациями
        sample_files: если указано, обрабатывает только эти файлы (для теста)

    Returns:
        dict: {file_id: {'audio': np.array, 'bounds': [[start1, end1], ...]}}
    """

    # Загружаем JSON
    with open(json_path, 'r') as f:
        word_bounds = json.load(f)
    print(len(word_bounds))

    # Cписок OPUS файлов
    opus_files = [f for f in os.listdir(base_path) if f.endswith('.opus')]
    if sample_files:
        opus_files = opus_files[:sample_files]
    print(f"Найдено {len(opus_files)} OPUS файлов")

    # Конвертируем каждый файл
    audio_data = {}

    for opus_file in tqdm(opus_files, desc="Конвертация OPUS -> WAV тензор"):
        if opus_file.startswith('._'):
            continue
        try:

            file_id = opus_file.replace('.opus', '')
            opus_path = os.path.join(base_path, opus_file)

            audio_array = librosa.load(opus_path, sr=TARGET_SR, mono=True)[0]

            bounds_list = word_bounds.get(file_id, [])

            audio_data[file_id] = {
                'audio': audio_array,
                'bounds': bounds_list,
                'duration': len(audio_array) / TARGET_SR,
                'has_word': len(bounds_list) > 0
            }

        except Exception as e:
            print(f"Ошибка при обработке {opus_file}: {e}")
            continue

    print(f"✅ Успешно обработано {len(audio_data)} файлов")
    print(f"   - Файлов с целевым словом: {sum(1 for d in audio_data.values() if d['has_word'])}")
    print(f"   - Файлов без целевого слова: {sum(1 for d in audio_data.values() if not d['has_word'])}")

    return audio_data

In [6]:
final_train = load_and_convert_opus_files(base_train_path, json_train_path)

NameError: name 'load_and_convert_opus_files' is not defined

Then we can convert them into correctly trimmed WAV files and then store them in the appropriate folders in the ```DATASET_DIR```.
We will use Pete's extract_loudest_section tool which you can find more documentation about here: https://github.com/petewarden/extract_loudest_section

In [2]:
# convert the ogg files to wavs
!mkdir wavs
!find *.ogg -print0 | xargs -0 basename -s .ogg | xargs -I {} ffmpeg -i {}.ogg -ar 16000 wavs/{}.wav
!rm -r -f *.ogg

# then use pete's tool to only extract 1 second clips from them for use with the KWS pipeline
!mkdir trimmed_wavs
!git clone https://github.com/petewarden/extract_loudest_section.git
!make -C extract_loudest_section/
!/tmp/extract_loudest_section/gen/bin/extract_loudest_section 'wavs/*.wav' trimmed_wavs/
!rm -r -f /wavs

find: ‘*.ogg’: No such file or directory
basename: missing operand
Try 'basename --help' for more information.
Cloning into 'extract_loudest_section'...
remote: Enumerating objects: 58, done.
remote: Total 58 (delta 0), reused 0 (delta 0), pack-reused 58 (from 1)
Receiving objects: 100% (58/58), 49.00 KiB | 111.00 KiB/s, done.
Resolving deltas: 100% (17/17), done.
make: Entering directory '/content/extract_loudest_section'
gcc --std=c++11 -O3 -DNDEBUG -I. -c main.cc -o /tmp/extract_loudest_section//gen/obj/./main.o
gcc --std=c++11 -O3 -DNDEBUG -I. -c status.cc -o /tmp/extract_loudest_section//gen/obj/./status.o
gcc --std=c++11 -O3 -DNDEBUG -I. -c wav_io.cc -o /tmp/extract_loudest_section//gen/obj/./wav_io.o
gcc  -I. \
-o /tmp/extract_loudest_section//gen/bin//extract_loudest_section /tmp/extract_loudest_section//gen/obj/./main.o /tmp/extract_loudest_section//gen/obj/./status.o /tmp/extract_loudest_section//gen/obj/./wav_io.o \
 -lstdc++ -lm
make: Leaving directory '/content/extract_lou

In [3]:
# Store them in the appropriate folders
data_index = {}
os.chdir('trimmed_wavs')
search_path = os.path.join('*.wav')
for wav_path in glob.glob(search_path):
    original_wav_path = wav_path
    parts = wav_path.split('_')
    if len(parts) > 2:
        wav_path = parts[0] + '_' + ''.join(parts[1:])
    matches = re.search('([^/_]+)_([^/_]+)\.wav', wav_path)
    if not matches:
        raise Exception('File name not in a recognized form:"%s"' % wav_path)
    word = matches.group(1).lower()
    instance = matches.group(2).lower()
    if not word in data_index:
      data_index[word] = {}
    if instance in data_index[word]:
        raise Exception('Audio instance already seen:"%s"' % wav_path)
    data_index[word][instance] = original_wav_path

output_dir = os.path.join('..', 'dataset')
try:
    os.mkdir(output_dir)
except:
    pass
for word in data_index:
  word_dir = os.path.join(output_dir, word)
  try:
      os.mkdir(word_dir)
      print('Created dir: ' + word_dir)
  except:
      print('Storing in existing dir: ' + word_dir)
  for instance in data_index[word]:
    wav_path = data_index[word][instance]
    output_path = os.path.join(word_dir, instance + '.wav')
    shutil.copyfile(wav_path, output_path)
os.chdir('..')
!rm -r -f trimmed_wavs

<>:10: SyntaxWarning: invalid escape sequence '\.'
<>:10: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipython-input-742518429.py:10: SyntaxWarning: invalid escape sequence '\.'
  matches = re.search('([^/_]+)_([^/_]+)\.wav', wav_path)


NameError: name 'os' is not defined

If you would like to download your full custom dataset as a zipfile for use later you can run the commented out cell below. And then download it from the Files area by clicking on the three dots next to the zip file. Note: this command will take a little while to run as the combination of your data and Pete's dataset will be relatively large!

If you would like to create a zip of just your dataset please re-run this colab (or factory reset it to wipe the memory) and skip the line that downloads Pete's dataset. That said, we STRONGLY suggest you do use Pete's dataset for any actual training you do!

Finally if you have room in your Google Drive and would like to load and store your dataset from there in the future you can mount your Google Drive in Colab [as described in this blog post](https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166).

In [ ]:
#!zip -r myKWSDataset.zip dataset

### Configure Your Model!
Ok now that your custom dataset is all ready to go we'll need to select your keywords and model settings with which to train!

```WANTED_WORDS``` = A comma-delimited string of the words you want to train for (e.g., "yes,no").

**Make sure to input the keywords you collected!**

In [ ]:
WANTED_WORDS = "yes,no"

The number of training steps and learning rates can be specified as comma-separated strings to define the amount/rate at each stage. For example, ```TRAINING_STEPS="12000,3000"``` and ```LEARNING_RATE="0.001,0.0001"``` will run 12,000 training steps with a rate of 0.001 followed by 3,000 final steps with a learning rate of 0.0001. These are good default values to work off of when you choose your values as the course staff has gotten this to work well with those values in the past!

In [ ]:
TRAINING_STEPS = "6000,1000"
LEARNING_RATE = "0.001,0.0001"

We suggest you leave the ```MODEL_ARCHITECTURE``` as tiny_conv the first time but if you would like to do this again and explore additional models some options are: ```single_fc, conv, low_latency_conv, low_latency_svdf, tiny_embedding_conv```. **Do remember if you switch the model type you may need to update the C++ code to include the ```tflite::AllOpsResolver``` to make sure you have all of the neccessary ops!**

In [ ]:
MODEL_ARCHITECTURE = 'tiny_conv'

In [ ]:
# Calculate the total number of steps, which is used to identify the checkpoint
# file name.
TOTAL_STEPS = str(sum(map(lambda string: int(string), TRAINING_STEPS.split(","))))

# Print the configuration to confirm it
print("Training these words: %s" % WANTED_WORDS)
print("Training steps in each stage: %s" % TRAINING_STEPS)
print("Learning rate in each stage: %s" % LEARNING_RATE)
print("Total number of training steps: %s" % TOTAL_STEPS)

**We suggest that you do not modify** the following constants as they include filepaths used in this notebook and data that is shared during training and inference.

In [ ]:
# Calculate the percentage of 'silence' and 'unknown' training samples required
# to ensure that we have equal number of samples for each label.
number_of_labels = WANTED_WORDS.count(',') + 1
number_of_total_labels = number_of_labels + 2 # for 'silence' and 'unknown' label
equal_percentage_of_training_samples = int(100.0/(number_of_total_labels))
SILENT_PERCENTAGE = equal_percentage_of_training_samples
UNKNOWN_PERCENTAGE = equal_percentage_of_training_samples

# Constants used during training only
VERBOSITY = 'DEBUG'
EVAL_STEP_INTERVAL = '1000'
SAVE_STEP_INTERVAL = '1000'

# Constants for training directories and filepaths
LOGS_DIR = 'logs/'
TRAIN_DIR = 'train/' # for training checkpoints and other files.

# Constants for inference directories and filepaths
import os
MODELS_DIR = 'models'
if not os.path.exists(MODELS_DIR):
  os.mkdir(MODELS_DIR)
MODEL_TF = os.path.join(MODELS_DIR, 'KWS_custom.pb')
MODEL_TFLITE = os.path.join(MODELS_DIR, 'KWS_custom.tflite')
FLOAT_MODEL_TFLITE = os.path.join(MODELS_DIR, 'KWS_custom_float.tflite')
MODEL_TFLITE_MICRO = os.path.join(MODELS_DIR, 'KWS_custom.cc')
SAVED_MODEL = os.path.join(MODELS_DIR, 'KWS_custom_saved_model')

**Be careful if you modify** the following constants as they will have downstream effects on the C++ code which you will then have to change. This mainly relate to hyperparaemeters for quantization and preprocessing. The first time you train a custom model **we suggest you do not modify these as well.**

In [ ]:
# Constants which are shared during training and inference
PREPROCESS = 'micro'
WINDOW_STRIDE = 20

# Constants for Quantization
QUANT_INPUT_MIN = 0.0
QUANT_INPUT_MAX = 26.0
QUANT_INPUT_RANGE = QUANT_INPUT_MAX - QUANT_INPUT_MIN

# Constants for audio process during Quantization and Evaluation
SAMPLE_RATE = 16000
CLIP_DURATION_MS = 1000
WINDOW_SIZE_MS = 30.0
FEATURE_BIN_COUNT = 40
BACKGROUND_FREQUENCY = 0.8
BACKGROUND_VOLUME_RANGE = 0.1
TIME_SHIFT_MS = 100.0

# Use the custom local dataset and set the tes/val/train split
DATA_URL = ''
VALIDATION_PERCENTAGE = 10
TESTING_PERCENTAGE = 10

# Train the model

### Load in TensorBoard to visulaize the training process.

As training progresses you should see the training status show up in the Tensorboard area. If this works it is very helpful for analyzing your training progress. Unfortunately, the staff has found that it sometimes doesn't start showing data for a while (~15 minutes) and sometimes doesn't show data until training completes (and instead shows ```No dashboards are active for the current data set```.). If it is working and then stops updating look to the top of the cell and click reconnect.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {LOGS_DIR}

### Launch Training

**ARE YOU USING A GPU INSTANCE!?!?!**
Make sure to check ```runtime->change runtime type``` as a GPU runtime will take ~2 hours to complete training whereas a CPU runtime with take ~10 hours!

**Importantly, every hour or so you should make sure to remind colab you are still there by clicking around in a cell or clicking reconnect on the tensorboard cell (after 90 minutes of no activity Colab may kill the instance)!** But/and beyond that you can absolutely minimize the window and continue with other work.

**If you run training a second time** we suggest restarting your runtime ```runtime->restart runtime``` in order to ensure all caches are cleared. However, do note that doing so will clear all cells and so you will have to run the entire file again.

If you would like to get more information on the training script you can find the source code for the script [here](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/speech_commands/train.py). In short it sets up the optimizer and preprocessor based on all of the flags we pass in!

Finally, by setting the ```VERBOSITY = 'DEBUG'``` above be aware that the training cell will print A LOT of information. Specifically you will get the accuracy and loss at each step as well as a confusion matrix every 1000 steps. We hope that is helpful in case TensorBoard fails to work. If you would like to run with less printouts you can change the setting to ```WARN``` or ```FATAL```. You will find this in the "Configure Your Model!" section.

In [ ]:
!python tensorflow/tensorflow/examples/speech_commands/train.py \
  --data_dir={DATASET_DIR} \
  --data_url={DATA_URL} \
  --wanted_words={WANTED_WORDS} \
  --silence_percentage={SILENT_PERCENTAGE} \
  --unknown_percentage={UNKNOWN_PERCENTAGE} \
  --preprocess={PREPROCESS} \
  --window_stride={WINDOW_STRIDE} \
  --model_architecture={MODEL_ARCHITECTURE} \
  --how_many_training_steps={TRAINING_STEPS} \
  --learning_rate={LEARNING_RATE} \
  --train_dir={TRAIN_DIR} \
  --summaries_dir={LOGS_DIR} \
  --verbosity={VERBOSITY} \
  --eval_step_interval={EVAL_STEP_INTERVAL} \
  --save_step_interval={SAVE_STEP_INTERVAL}

# Generating your Model
Just like with the pre-trained model we will now take the final checkpoint and convert it into a quantized TensorFlow Lite model.

### Generate a TensorFlow Model for Inference

Combine relevant training results (graph, weights, etc) into a single file for inference. This process is known as freezing a model and the resulting model is known as a frozen model/graph, as it cannot be further re-trained after this process.

In [ ]:
!rm -rf {SAVED_MODEL}
!python tensorflow/tensorflow/examples/speech_commands/freeze.py \
--wanted_words=$WANTED_WORDS \
--window_stride_ms=$WINDOW_STRIDE \
--preprocess=$PREPROCESS \
--model_architecture=$MODEL_ARCHITECTURE \
--start_checkpoint=$TRAIN_DIR$MODEL_ARCHITECTURE'.ckpt-'{TOTAL_STEPS} \
--save_format=saved_model \
--output_file={SAVED_MODEL}

### Generate a TensorFlow Lite Model

Convert the frozen graph into a TensorFlow Lite model, which is fully quantized for use with embedded devices. The following cell will also print the model size.

In [ ]:
model_settings = models.prepare_model_settings(
    len(input_data.prepare_words_list(WANTED_WORDS.split(','))),
    SAMPLE_RATE, CLIP_DURATION_MS, WINDOW_SIZE_MS,
    WINDOW_STRIDE, FEATURE_BIN_COUNT, PREPROCESS)
audio_processor = input_data.AudioProcessor(
    DATA_URL, DATASET_DIR,
    SILENT_PERCENTAGE, UNKNOWN_PERCENTAGE,
    WANTED_WORDS.split(','), VALIDATION_PERCENTAGE,
    TESTING_PERCENTAGE, model_settings, LOGS_DIR)

**Note: if the below cell fails it might be because you do not have enough data to have 100 recordings in the representative dataset!** If this happens you will see an error that says something like ```ValueError: cannot reshape array of size 0 into shape (1,1960)```. To help you fix this we have added a ```print(i)``` into the loop. As such, all you have to do is change the ```REP_DATA_SIZE``` variable to be equal to the last integer value printed out by the loop and then re-run the cell!

In [ ]:
REP_DATA_SIZE = 100
with tf.Session() as sess:
  # with tf.compat.v1.Session() as sess: #replaces the above line for use with TF2.x
  float_converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL)
  float_tflite_model = float_converter.convert()
  float_tflite_model_size = open(FLOAT_MODEL_TFLITE, "wb").write(float_tflite_model)
  print("Float model is %d bytes" % float_tflite_model_size)

  converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL)
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  converter.inference_input_type = tf.lite.constants.INT8
  # converter.inference_input_type = tf.compat.v1.lite.constants.INT8 #replaces the above line for use with TF2.x
  converter.inference_output_type = tf.lite.constants.INT8
  # converter.inference_output_type = tf.compat.v1.lite.constants.INT8 #replaces the above line for use with TF2.x
  def representative_dataset_gen():
    for i in range(REP_DATA_SIZE):
      data, _ = audio_processor.get_data(1, i*1, model_settings,
                                         BACKGROUND_FREQUENCY,
                                         BACKGROUND_VOLUME_RANGE,
                                         TIME_SHIFT_MS,
                                         'testing',
                                         sess)
      flattened_data = np.array(data.flatten(), dtype=np.float32).reshape(1, 1960)
      print(i)
      yield [flattened_data]
  converter.representative_dataset = representative_dataset_gen
  tflite_model = converter.convert()
  tflite_model_size = open(MODEL_TFLITE, "wb").write(tflite_model)
  print("Quantized model is %d bytes" % tflite_model_size)


### Testing the accuracy after Quantization

Verify that the model we've exported is still accurate, using the TF Lite Python API and our test set.

In [ ]:
# Helper function to run inference
def run_tflite_inference_testSet(tflite_model_path, model_type="Float"):
  #
  # Load test data
  #
  np.random.seed(0) # set random seed for reproducible test results.
  with tf.Session() as sess:
    # with tf.compat.v1.Session() as sess: #replaces the above line for use with TF2.x
    test_data, test_labels = audio_processor.get_data(
        -1, 0, model_settings, BACKGROUND_FREQUENCY, BACKGROUND_VOLUME_RANGE,
        TIME_SHIFT_MS, 'testing', sess)
  test_data = np.expand_dims(test_data, axis=1).astype(np.float32)

  #
  # Initialize the interpreter
  #
  interpreter = tf.lite.Interpreter(tflite_model_path)
  interpreter.allocate_tensors()
  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  #
  # For quantized models, manually quantize the input data from float to integer
  #
  if model_type == "Quantized":
    input_scale, input_zero_point = input_details["quantization"]
    test_data = test_data / input_scale + input_zero_point
    test_data = test_data.astype(input_details["dtype"])

  #
  # Evaluate the predictions
  #
  correct_predictions = 0
  for i in range(len(test_data)):
    interpreter.set_tensor(input_details["index"], test_data[i])
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]
    top_prediction = output.argmax()
    correct_predictions += (top_prediction == test_labels[i])

  print('%s model accuracy is %f%% (Number of test samples=%d)' % (
      model_type, (correct_predictions * 100) / len(test_data), len(test_data)))

In [ ]:
# Compute float model accuracy
run_tflite_inference_testSet(FLOAT_MODEL_TFLITE)

# Compute quantized model accuracy
run_tflite_inference_testSet(MODEL_TFLITE, model_type='Quantized')

### Generate a TensorFlow Lite for Microcontrollers Model
To convert the TensorFlow Lite quantized model into a C source file that can be loaded by TensorFlow Lite for Microcontrollers on Arduino we simply need to use the ```xxd``` tool to convert the ```.tflite``` file into a ```.cc``` file (just like we did in the pervious section).

In [ ]:
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

That's it! You've successfully converted your TensorFlow Lite model into a TensorFlow Lite for Microcontrollers model! Run the cell below to print out its contents which we'll need for our next step, deploying the model using the Arudino IDE!

In [ ]:
!cat {MODEL_TFLITE_MICRO}

To download your model for use at a later date:

1. On the left of the UI click on the folder icon
2. Click on the three dots to the right of the ```.cc``` file you just generated and select "download." The file can be found at ```models/{MODEL_TFLITE_MICRO}``` which by default is ```models/KWS_custom.cc```

Next we'll deploy that model using the Arduino IDE.